In [1]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr, lit
from pyspark.sql.window import Window
import duckdb

In [2]:
spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

24/04/16 18:31:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# compravenda_sup
DF = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:formatted_zone/compravenda_sup.db") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .option("query", "SELECT * FROM RDD") \
    .load()

# verificar duplicats
print(f"Hi ha {DF.count() - DF.distinct().count()} valors duplicats al DataFrame 'formatted_zone/compravenda_sup.db' de {DF.count()} valors totals")

# substituir "--" por NA
DF = DF.withColumn('Nombre', F.when(col('Nombre') == '--', None).otherwise(col('Nombre')))
# verificar NA
print(f'Hi ha {DF.count() - DF.na.drop().count()} NAs')
# imprimir toda la informació que té un NA
valorsnull = DF.filter(DF.Nombre.isNull())

# passar a numeric
DF = DF.withColumn('Nombre', DF['Nombre'].cast('float'))

# Imputar NA de Nombre amb la mitjana de "Nombre" per "Nom_barri"
mean_nombre_nom_barri = DF.groupBy('Nom_barri').agg(F.mean('Nombre').alias('mean_nombre_nom_barri'))

DF = DF.join(mean_nombre_nom_barri, on='Nom_barri', how='left')
DF = DF.withColumn('Nombre', F.when(DF['Nombre'].isNull(), DF['mean_nombre_nom_barri']).otherwise(DF['Nombre']))
# Verificar NAs després de la imputació per barri
print(f'Hi ha {DF.count() - DF.na.drop().count()} NAs en barri')

# Imputar NA de Nombre amb la mitjana de "Nombre" per "Nom_districte"
mean_nombre_nom_districte = DF.groupBy('Nom_Districte').agg(F.mean('Nombre').alias('mean_nombre_nom_districte'))

DF = DF.join(mean_nombre_nom_districte, on='Nom_Districte', how='left')
DF = DF.withColumn('Nombre', F.when(DF['Nombre'].isNull(), DF['mean_nombre_nom_districte']).otherwise(DF['Nombre']))

# Verificar NAs després de la imputació per districte
print(f'Hi ha {DF.count() - DF.na.drop().count()} NAs en districte')

# Imprimir totes les files amb NA a la columna "Nombre"
DF.filter(DF.Nombre.isNull()).show()

min_value = DF.groupBy('Superfície_mitjana_(m2_construïts)').agg(F.min('Nombre').alias('min_value'))
max_value = DF.groupBy('Superfície_mitjana_(m2_construïts)').agg(F.max('Nombre').alias('max_value'))

DF = DF.join(min_value, on='Superfície_mitjana_(m2_construïts)', how='left')
DF = DF.join(max_value, on='Superfície_mitjana_(m2_construïts)', how='left')

DF = DF.withColumn('Nombre', F.when(DF['Superfície_mitjana_(m2_construïts)'] == 0, 0).otherwise(DF['Nombre']-DF['min_value'])/(DF['max_value']-DF['min_value']))

# outliers
IQR = DF.approxQuantile('Nombre', [0.25, 0.75], 0.001)
IQR_values = IQR[1] - IQR[0]
        
DF = DF.withColumn('IQR', lit(IQR_values))
                
not_outliers = DF.filter(col('Nombre') >= IQR[0] - 1.5 * IQR_values)\
            .filter(col('Nombre') <= IQR[1] + 1.5 * IQR_values)

outliers = DF.subtract(not_outliers)
print(f"Hi ha {outliers.count()} outliers detectats per quartils")
  
# eliminar columnes auxiliars
DF = DF.drop('min_value', 'max_value', 'mean_nombre_nom_barri', 'mean_nombre_nom_districte', 'IQR')
  

Hi ha 0 valors duplicats al DataFrame 'formatted_zone/compravenda_sup.db' de 876 valors totals
Hi ha 290 NAs
Hi ha 12 NAs en barri
Hi ha 12 NAs en districte
+-------------+---------+---+---------+--------------+----------+----------------------------------+------+---------------------+-------------------------+
|Nom_Districte|Nom_Barri|Any|Trimestre|Codi_Districte|Codi_Barri|Superfície_mitjana_(m2_construïts)|Nombre|mean_nombre_nom_barri|mean_nombre_nom_districte|
+-------------+---------+---+---------+--------------+----------+----------------------------------+------+---------------------+-------------------------+
+-------------+---------+---+---------+--------------+----------+----------------------------------+------+---------------------+-------------------------+



Hi ha 79 outliers detectats per quartils


In [4]:
DF.show()

+----------------------------------+--------------+--------------------+----+---------+--------------+----------+-------------------+
|Superfície_mitjana_(m2_construïts)| Nom_Districte|           Nom_Barri| Any|Trimestre|Codi_Districte|Codi_Barri|             Nombre|
+----------------------------------+--------------+--------------------+----+---------+--------------+----------+-------------------+
|                             Total|     Les Corts|           les Corts|2023|        1|             4|        19|0.16117216998719475|
|                             Total|     Les Corts|la Maternitat i S...|2023|        1|             4|        20|0.20227920475625832|
|                             Total|     Les Corts|           Pedralbes|2023|        1|             4|        21|0.42328043749190036|
|                             Total|Sants-Montjuïc|   la Marina de Port|2023|        1|             3|        13|0.11477411752288869|
|                             Total|Sants-Montjuïc|la Marina d

In [5]:
spark.stop()